# Practica cinco

Grupo 14:
* Joaquín Ibáñez Penalva
* Aurora Zuoris

Para la realización de esta práctica se usará la librería de numpy, pandas, matplotlib, y sklearn.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

## Ejercicio 1

In [ ]:
class Perceptron:
  def __init__(self, max_iter=1000, mezclar=True, eta=1.0, random_state=42): # max_iter: cantidad máxima de iteraciones, mezclar: si se mezclan los datos, eta: tasa de aprendizaje, random_state: semilla para el generador de números aleatorios
    self._max_iter = max_iter
    self._mezclar = mezclar
    self._eta = eta
    self._random_state = random_state

  def ajustar(self, X, y): # X: matriz bidimensional, y: vector
    if len(X.shape) != 2: # si X no es bidimensional
      raise ValueError("X debe ser una matriz bidimensional") # lanza una excepción
    if len(y.shape) != 1: # si y no es un vector
      raise ValueError("y debe ser un vector") # lanza una excepción
    if X.shape[0] != y.shape[0]: # si X y y no tienen la misma cantidad de filas
      raise ValueError("X y y deben tener la misma cantidad de filas") # lanza una excepción
    
    if len(np.unique(y)) != 2: # si y no tiene dos clases
      raise ValueError("y debe tener dos clases") # lanza una excepción
    
    self._label_to_code = {label: code for code, label in enumerate(np.unique(y))} # crea un diccionario con las etiquetas de y y sus codificaciones
    self._code_to_label = {code: label for label, code in self._label_to_code.items()} # crea un diccionario con las codificaciones de y y sus etiquetas

    y = np.vectorize(self._label_to_code.get)(y) # convierte las etiquetas de y en sus codificaciones
    
    n, m = X.shape # n: cantidad de filas de X, m: cantidad de columnas de X

    weights = np.zeros(m) # crea un vector de ceros de tamaño m
    bias = 0 # crea un escalar cero

    for _ in range(self._max_iter): # itera hasta que se cumpla la condición
      if self._mezclar: # si mezclar es True
        X, y = self._shuffle(X, y) # mezcla X y y
      
      res = np.dot(X, weights) + bias # calcula el producto punto de X y weights y le suma bias (el escalar cero)
      res_sign = np.sign(res) # calcula el signo de res
      res_sign[res_sign == 0] = -1 # si el signo de res es cero, lo convierte en -1
      errors = ~((res_sign == 1) ^ (y == 1)) # calcula los errores
      if np.all(~errors): # si no hay errores
        break # termina el bucle
      errors_sign = np.where(errors, np.where(res_sign == 1, -1, 1), 0) # calcula el signo de los errores

      delta_bias = self._eta * np.sum(errors_sign) # calcula el cambio de bias
      delta_weights = self._eta * np.dot(errors_sign, X) # calcula el cambio de weights

      bias += delta_bias # actualiza bias
      weights += delta_weights # actualiza weights
    self.pesos_ = weights # guarda los pesos
    self.pesos_umbral_ = bias # guarda el peso umbral
    return self # devuelve el objeto
  
  def predecir(self, X): # X: matriz bidimensional
    y_code =  np.dot(X, self.pesos_) + self.pesos_umbral_ > 0 # calcula el producto punto de X y los pesos y le suma el peso umbral y compara si es mayor a cero
    y_code = np.where(y_code, 1, 0) # si y_code es True, lo convierte en 1, si es False, lo convierte en 0
    return np.vectorize(self._code_to_label.get)(y_code) # convierte las codificaciones de y_code en sus etiquetas
    

  def _shuffle(self, X, y): # mezcla X e y
    n, _ = X.shape # n: cantidad de filas de X, _: cantidad de columnas de X
    rng = np.random.RandomState(self._random_state) # crea un generador de números aleatorios
    idx = rng.permutation(n) # crea un vector con los índices de las filas de X en orden aleatorio
    
    if(type(X) == pd.DataFrame): # si X es un DataFrame
      X = X.to_numpy() # convierte X en un array de numpy
    return X[idx], y[idx] # devuelve X e y con las filas en orden aleatorio


## Ejercicio 2

In [ ]:
df = pd.read_csv('iris_pca_2d.csv')

df.head()

In [ ]:
# Se separan los datos segun las clases de primeras.
classes = set(df['clase'].unique())
split_classes = [classes - {c} for c in classes]

fig, axs = plt.subplots(3, 2, figsize=(10, 15))

pc1_range = np.linspace(df['pc1'].min(), df['pc1'].max(), 100)

for i, c in enumerate(split_classes):
  class_to_col = {cl: co for co, cl in zip(('red', 'blue'), c)}
  local_df = df[df['clase'].isin(c)]
  x_train, x_test, y_train, y_test = train_test_split(local_df[['pc1', 'pc2']], local_df['clase'], test_size=0.3, random_state=42)
  perceptron = Perceptron(max_iter=50_000).ajustar(x_train, y_train)

  line = - (perceptron.pesos_[0] / perceptron.pesos_[1]) * pc1_range - (perceptron.pesos_umbral_ / perceptron.pesos_[1])

  axs[i, 0].scatter(x_train['pc1'], x_train['pc2'], c=y_train.map(lambda x: class_to_col[x]), label=y_train.unique())
  axs[i, 1].scatter(x_test['pc1'], x_test['pc2'], c=y_test.map(lambda x: class_to_col[x]), label=y_test)

  axs[i, 0].plot(pc1_range, line, 'k-')
  axs[i, 1].plot(pc1_range, line, 'k-')

  axs[i, 0].set_ylim(df['pc2'].min(), df['pc2'].max())
  axs[i, 1].set_ylim(df['pc2'].min(), df['pc2'].max())

  legend_elems = [
    plt.Line2D([0], [0], marker='o', color='w', label=c, markerfacecolor=class_to_col[c], markersize=10)
    for c in c
  ]

  axs[i, 0].legend(handles=legend_elems)
  axs[i, 1].legend(handles=legend_elems)

  name = '-'.join(c)

  axs[i, 0].set_title(f'Entrenamiento {name}')
  axs[i, 1].set_title(f'Test {name}')